# Задача

- Прочитать файл
- Посчитать количество последовательностей в fasta файле;
- Добавить рассчет статистики, которую писали для строк для каждой последовательности, присутсвующей в fasta файле;
- Найти в последовательности стар и стоп кодоны и записать их координанты в последовательности;
- если это ДНК: сделать транскрипцию, а затем трансляцию, причем для случая 5'-3' и 3'-5'. В случае если это РНК, сделать только трансляцию;
- Основываясь на таблице аминокислот посчитать заряд полученной амминокислотной последовательности;
- Добавить возможность записи всей статистики в файл для пользователя, а также записывать в файл результат транскрипиции, трансляции;



In [89]:
import re
import os
path_to_code = os.getcwd()
path_to_file = os.path.join(path_to_code, os.pardir, 'lections', 'files_and_images', 'wombat_mit.fasta')
   
#---------------------------------------------------------------    
    
def read_fasta_file(inp_fasta_file):
    name, seq = '', []
    for line in inp_fasta_file:
        line = line.rstrip()
        if line.startswith(">"):
            if name:
                yield(name, ''.join(seq))
            name, seq = line, []
        else: 
            seq.append(line)
    if name: 
        yield(name, ''.join(seq))
        
#---------------------------------------------------------------
        
def nucl_seq_type(nucl_seq):
    par = False
    if 'T' in list(nucl_seq): par = True
    return par

def nucl_seq_mean_value(nucl_seq):
    mean_value = (list(nucl_seq).count('A') + list(nucl_seq).count('G') + list(nucl_seq).count('C') + list(nucl_seq).count('T') + list(nucl_seq).count('U')) / 4
    return mean_value

def nucl_seq_count(nucl_seq):
    dict_of_nucl = {}
    if nucl_seq_type(nucl_seq):
        for elem in list('ATGC'):
            dict_of_nucl.update([(elem, list(nucl_seq).count(elem))])
    else:
        for elem in list('AUGC'):
            dict_of_nucl.update([(elem, list(nucl_seq).count(elem))])
    return dict_of_nucl

def nucl_seq_gc(nucl_seq):
    dict_of_nucl = nucl_seq_count(nucl_seq)
    gc_composition = ((dict_of_nucl['G'] + dict_of_nucl['C']) / len(nucl_seq)) * 100
    return gc_composition

def nucl_seq_complement(nucl_seq):
    nucl_seq_comp = []
    for elem in nucl_seq:
        if elem == 'A': nucl_seq_comp.append('T')
        if elem == 'T': nucl_seq_comp.append('A')
        if elem == 'G': nucl_seq_comp.append('C')
        if elem == 'C': nucl_seq_comp.append('G')
    return ''.join(nucl_seq_comp)

def nucl_seq_second_chain(nucl_seq):
    if nucl_seq_type(nucl_seq): nucl_seq_sec = nucl_seq_complement(nucl_seq)[ : :-1]
    else: nucl_seq_sec = 'Not a DNA'
    return nucl_seq_sec

def nucl_seq_stat(nucl_seq):
    print('STATS:')
    print(f'Initial sequence: {nucl_seq}')
    if nucl_seq_type(nucl_seq): print('This is a DNA sequence')
    else: print('This is a RNA sequence')
    print(f'Mean value = {nucl_seq_mean_value(nucl_seq)}')
    print(f'Length = {len(nucl_seq)}')
    print(nucl_seq_count(nucl_seq))
    print(f'GC - composition = {nucl_seq_gc(nucl_seq)}%')
    print(f'Second DNA chain: {nucl_seq_second_chain(nucl_seq)}')
    
def nucl_seq_stat_to_dic(nucl_seq):
    seq_stat_dic = {}
    seq_stat_dic['seq'] = nucl_seq
    if nucl_seq_type(nucl_seq): seq_stat_dic['type'] = 'DNA'
    else: seq_stat_dic['type'] = 'RNA'
    seq_stat_dic['mean'] = nucl_seq_mean_value(nucl_seq)
    seq_stat_dic['length'] = len(nucl_seq)
    seq_stat_dic['count'] = nucl_seq_count(nucl_seq)
    seq_stat_dic['gs'] = nucl_seq_gc(nucl_seq)
    seq_stat_dic['sec_chain'] = nucl_seq_second_chain(nucl_seq)    
    return seq_stat_dic

def double_nucl_seq_stat_to_dic(first_nucl_seq):
    second_nucl_seq = nucl_seq_second_chain(first_nucl_seq)
    double_seq_stat_dic = {}
    double_seq_stat_dic['5\'-3\''] = nucl_seq_stat_to_dic(first_nucl_seq)
    double_seq_stat_dic['3\'-5\''] = nucl_seq_stat_to_dic(second_nucl_seq)  
    return double_seq_stat_dic        
    
#---------------------------------------------------------------   

def nucl_seq_transcription(nucl_seq):
    if nucl_seq_type(nucl_seq): 
        nucl_seq_transcr = nucl_seq_complement(nucl_seq).replace('T','U')
    else: nucl_seq_transcr = 'Need DNA for transcription'
    return nucl_seq_transcr

def nucl_seq_translation(nucl_seq):
    amino_seq = []
    if not nucl_seq_type(nucl_seq):
        for i in range(0, len(nucl_seq) - 3, 3):
            amino_seq.append(gene_code_dic[nucl_seq[i:i+3]])
    else: amino_seq.append('Need RNA for translation')
    return ''.join(amino_seq)    

def nucl_seq_start_codons_indexes(nucl_seq):
    regexp_start_codons = re.compile('AUG')
    start_codons_indexes = []
    index = 0
    if regexp_start_codons.search(nucl_seq):
        while regexp_start_codons.search(nucl_seq[index:]):
            if regexp_start_codons.search(nucl_seq[index:]).start() % 3 == 0: 
                start_codons_indexes.append(index + regexp_start_codons.search(nucl_seq[index:]).start())
                index += regexp_start_codons.search(nucl_seq[index:]).end()
            elif (regexp_start_codons.search(nucl_seq[index:]).start() - 1) % 3 == 0: index += regexp_start_codons.search(nucl_seq[index:]).start() + 2
            else: index += regexp_start_codons.search(nucl_seq[index:]).start() + 1
    return start_codons_indexes

def nucl_seq_stop_codons_indexes(nucl_seq):
    regexp_stop_codons = re.compile('UA[AG]|UGA')
    stop_codons_indexes = []
    index = 0
    if regexp_stop_codons.search(nucl_seq):
        while regexp_stop_codons.search(nucl_seq[index:]):
            if regexp_stop_codons.search(nucl_seq[index:]).start() % 3 == 0: 
                stop_codons_indexes.append(index + regexp_stop_codons.search(nucl_seq[index:]).start())
                index += regexp_stop_codons.search(nucl_seq[index:]).end()
            elif (regexp_stop_codons.search(nucl_seq[index:]).start() - 1) % 3 == 0: index += regexp_stop_codons.search(nucl_seq[index:]).start() + 2
            else: index += regexp_stop_codons.search(nucl_seq[index:]).start() + 1
    return stop_codons_indexes

def charge_of_amino_seq(amino_seq):
    charge = 0
    for char in amino_seq:
        if char in amino_acids_charge_dic.keys():
            charge += amino_acids_charge_dic[char]
    return charge

#---------------------------------------------------------------  

gene_code_dic = {'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L', 'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S', 'UAU': 'Y', 'UAC': 'Y', 'UAA': 'end', 'UAG': 'end', \
                'UGU' : 'C', 'UGC': 'C', 'UGA': 'end', 'UGG': 'W', 'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L', 'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P', \
                'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R', 'AUU': 'I', 'AUC': 'I', 'AUA': 'I', 'AUG': 'M', \
                'ACU': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T', 'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K', 'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R', \
                'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V', 'GCU': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A', 'GAU': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E', \
                'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'}

amino_acids_charge_dic = {'D': -1, 'E': -1, 'K': 1, 'R': 1, 'H': 1}

#---------------------------------------------------------------  

with open(path_to_file, 'r') as inp_fasta_file:
    par = 0
    for name, seq in read_fasta_file(inp_fasta_file):
        par += 1
        print(name, seq, sep = '\n', end = '\n\n')
        print('----------------------------------------------------------------------------------------')
        nucl_seq_stat(seq)
        print('----------------------------------------------------------------------------------------')
        
        if nucl_seq_type(seq):
            sec_seq = nucl_seq_second_chain(seq)
            first_seq_transcripted = nucl_seq_transcription(seq)
            print(f'5\'-3\' after transcription: {first_seq_transcripted}')
            start_codons_indexes = nucl_seq_start_codons_indexes(first_seq_transcripted)
            stop_codons_indexes = nucl_seq_stop_codons_indexes(first_seq_transcripted)
            print(f'Indexes of start codons: {start_codons_indexes}')
            print(f'Indexes of stop codons: {stop_codons_indexes}', end = '\n\n')    
            
            second_seq_transcripted = nucl_seq_transcription(sec_seq)
            print(f'3\'-5\' after transcription: {second_seq_transcripted}')
            start_codons_indexes = nucl_seq_start_codons_indexes(second_seq_transcripted)
            stop_codons_indexes = nucl_seq_stop_codons_indexes(second_seq_transcripted)
            print(f'Indexes of start codons: {start_codons_indexes}')
            print(f'Indexes of stop codons: {stop_codons_indexes}', end = '\n\n')    
            
            print('----------------------------------------------------------------------------------------')
            first_seq_transcripted_translated = nucl_seq_translation(first_seq_transcripted)
            print(f'5\'-3\' after transcription and translation: {first_seq_transcripted_translated}')
            print(f'Charge of amino sequence: {charge_of_amino_seq(first_seq_transcripted_translated)}', end = '\n\n')
            second_seq_transcripted_translated = nucl_seq_translation(second_seq_transcripted)
            print(f'5\'-3\' after transcription and translation: {second_seq_transcripted_translated}')
            print(f'Charge of amino sequence: {charge_of_amino_seq(second_seq_transcripted_translated)}', end = '\n\n')
            
    print('----------------------------------------------------------------------------------------')  
    print(f'\nNumber of sequences in fasta file: {par}')  

>NC_003322.1:14176-15321 Vombatus ursinus mitochondrion, complete genome
ATGATTAACCTACGCAAAACACATCCCCTTATAAAAATCGTCAATGAAGCATTCATCGACCTACCCACACCCTCCAATATCTCCGCCTGATGAAATTTTGGATCACTATTAGGAATCTGCCTTATCATACAAATCCTAACAGGCCTATTCCTAGCCATACATTACACCTCCGATACCCTAACAGCCTTCTCTTCAGTAGCCCATATCTGCCGAGATGTGAATCACGGCTGACTCATCCGCAACCTCCACGCCAACGGAGCGTCTATATTCTTCATATGCCTATACCTCCACATTGGCCGAGGAATCTATTATGGCTCCTACCTCTACAAAGAAACATGAAACATCGGAGTATTTCTTCTACTCACAGTTATAGCAACTGCTTTCGTTGGCTATGTACTCCCATGAGGACAAATATCCTTCTGAGGTGCAACCGTAATTACCAACTTATTATCAGCCATCCCTTACGTAGGCACCACCCTAGTAGAATGAATTTGAGGTGGATTCTCCGTAGACAAAGCCACACTGACCCGATTCTTCGCCTTCCATTTTATCCTACCCTTCATTGTCACAGCACTAGCTATCGTTCACCTACTATTCCTTCATGAAACAGGCTCAAACAACCCCTCAGGAATTAACCCCGACGCAGACAAAATTCCCTTCCACCCCTACTACACTACCAAAGACATTATAGGTGCAATCCTAATAATCCTTGCCCTCCTACTACTTACCCTATTCTCACCCGACATGTTAGGAGACCCAGACAACTTCTCCCCTGCCAACCCCCTCAGCACACCACCCCACATCAAACCAGAATGATATTTCCTATTCGCCTACGCTATTCTCCGATCAATCCCAAATAAACTGGGAGGAGTACTAGCCCTACTAGCATCCATCCTAATCCTCCTAGTCATCCCATTCCTGCATACA